# Introduction

Here I will give an overview of the basic functionality of `doctable`.

I will cover the following topics:

+ Connecting to the database using `ConnectCore`.
+ Defining a database schema using the `table_schema` decorator.
+ Creating the table using the `begin_ddl()` context manager.
+ Inserting values into the database using the `ConnectQuery` and `ConnectTable` interfaces.

In [1]:
import sys
sys.path.append('../')
import doctable
import pprint

The `ConnectCore` objects acts as the primary starting point for any actions performed on the database. We create a new connection to the datbase using the `.open()` factory method constructor.

In [2]:
core = doctable.ConnectCore.open(
    target=':memory:', 
    dialect='sqlite'
)
core

ConnectCore(target=':memory:', dialect='sqlite', engine=Engine(sqlite:///:memory:), metadata=MetaData())

Next we define a very basic schema using the `table_schema` decorator. This decorator is used to create a Container object, which contains information about the database schema and is also a dataclass that can be inserted or retrieved from the database. Read the schema definition examples for more information on creating container objects and database schemas.

In [3]:
@doctable.table_schema
class MyContainer0:
    id: int
    name: str
    age: int

doctable.inspect_schema(MyContainer0).column_info_df()

,Col Name,Col Type,Attr Name,Hint,Order,Primary Key,Foreign Key,Index,Default
0,id,Integer,id,int,"(inf, 0)",False,False,None,None
1,name,String,name,str,"(inf, 1)",False,False,None,None
2,age,Integer,age,int,"(inf, 2)",False,False,None,None


We actually connect to the database table using the context manager returned by `.begin_ddl()`. This design is necessary for multi-table schemas, but, because of the readability it provides, I will use it for single-table schemas as well. The method `create_table_if_not_exists` here returns a new instance of `DBTable`. Alternatively, we could reflect a database table, in which we would not be required to provide a schema container.

In [4]:
with core.begin_ddl() as emitter:
    tab0 = emitter.create_table_if_not_exists(container_type=MyContainer0)
for ci in core.inspect_columns('MyContainer0'):
    print(ci)

{'name': 'id', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'age', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}


Among other things, the table object allows you to specify particular colums, and these will be used when performing certain types of queries. The following methods can all be used to specify columns.

In [5]:
tab0['id']

Column('id', Integer(), table=<MyContainer0>)

In [6]:
tab0['id','age']

[Column('id', Integer(), table=<MyContainer0>),
 Column('age', Integer(), table=<MyContainer0>)]

In [7]:
tab0[:]

[Column('id', Integer(), table=<MyContainer0>),
 Column('name', String(), table=<MyContainer0>),
 Column('age', Integer(), table=<MyContainer0>)]

You may also use a context manager to manage shorthand versions of tables for more compact query code.

In [8]:
with tab0 as t:
    print(t)

DBTable(table=Table('MyContainer0', MetaData(), Column('id', Integer(), table=<MyContainer0>), Column('name', String(), table=<MyContainer0>), Column('age', Integer(), table=<MyContainer0>), schema=None), core=ConnectCore(target=':memory:', dialect='sqlite', engine=Engine(sqlite:///:memory:), metadata=MetaData()), schema=TableSchema(table_name='MyContainer0', container_type=<class '__main__.MyContainer0'>, columns=[ColumnInfo(attr_name='id', type_hint=<class 'int'>, defined_order=0, column_args=ColumnArgs(order=inf, column_name=None, type_kwargs={}, use_type=None, sqlalchemy_type=None, autoincrement=False, nullable=True, unique=None, primary_key=False, index=None, foreign_key=None, default=None, onupdate=None, server_default=None, server_onupdate=None, comment=None, other_kwargs={})), ColumnInfo(attr_name='name', type_hint=<class 'str'>, defined_order=1, column_args=ColumnArgs(order=inf, column_name=None, type_kwargs={}, use_type=None, sqlalchemy_type=None, autoincrement=False, nullabl

We can perform queries on the database using the `ConnectQuery` interface returned from the `ConnectCore.query()` method. In this case, we insert a new row into the database using the `insert_multi()` method. Note that we will use an alternative interface for inserting container instances into the database, as I will show later.

In [9]:
core.query().insert_multi(
    dtable = tab0, 
    data = [
        {'name': 'Devin J. Cornell', 'age': 50},
        {'name': 'Dorothy Andrews', 'age': 49},
    ]
)

The `select()` query works in much the same way. We simply specify which columns from the DB that we would like to retrieve. This select query returns instances of raw `sqlalchemy.engine.row.Row` objects from the sqlalchemy core interface. Use the `.all()` method to get a list of these rows, but see the select method documentation for a complete list.

In [10]:
core.query().select(tab0[:]).all()

[(None, 'Devin J. Cornell', 50), (None, 'Dorothy Andrews', 49)]

For example, we can use `scalars()` instead to get a sequence of single values. Note that I used `tab0['id',]` instead of `tab0['id']` because it returns a list of columns that is required for every select query.

In [11]:
list(core.query().select(tab0['id',]).scalars())

[None, None]

A more elegant approach is to use a context manager to make cleaner queries - especially when there are many parameters and the table must be referenced multiple times.

In [12]:
with tab0 as t, core.query() as q:
    ids = q.select(t['id',], where = t['id'] != None).scalars()
list(ids)

[]

The table `DBTable` interface also has a `.query()` method for when working with container objects directly is preferable. To insert container object instances into the table, I instead use the `DBTable.query()` method to generate a `TableQuery` instance. This behaves much like `ConnectQuery` except that returned data will be placed into new container instances and we may insert data from container instances directly.

In [13]:
tab0.query().insert_single(
    container_object = MyContainer0(id=16, name='John Doe', age=30),
)

In [14]:
tab0.query().insert_multi(
    data = [
        MyContainer0(id=0, name='Jane Doe', age=30),
        MyContainer0(id=0, name='Markus Morn', age=50),
    ],
)

Here I define a more complicated schema. 

+ The standard `id` column is now included. Notice that `order=0` means the column will appear first in the table.
+ The `updated` and `added` attributes have been created to automatically record the time of insertion and update.
+ I added the `birthyear` method to the container type.

In [15]:
import datetime
@doctable.table_schema(table_name='mytable1')
class MyContainer1:
    name: str
    age: int
    _id: int = doctable.Column(
        column_args=doctable.ColumnArgs(
            column_name = 'id',
            order=0, 
            primary_key=True, 
            autoincrement=True
        ),
    )
    updated: datetime.datetime = doctable.Column(
        column_args=doctable.ColumnArgs(default=datetime.datetime.utcnow),
    )
    added: datetime.datetime = doctable.Column(
        column_args=doctable.ColumnArgs(
            default=datetime.datetime.utcnow, 
            onupdate=datetime.datetime.utcnow,
        )
    )
    
    def birthyear(self):
        '''Retrieve the birthyear of the person at the time this database entry was added.'''
        try:
            return self.added.year - self.age
        except AttributeError as e:
            raise AttributeError('Cannot calculate birthyear without the added date. '
                'Did you mean to call this on a retrieved container instance?') from e
    
doctable.inspect_schema(MyContainer1).column_info_df()

,Col Name,Col Type,Attr Name,Hint,Order,Primary Key,Foreign Key,Index,Default
0,id,Integer,_id,int,"(0, 2)",True,False,None,None
1,name,String,name,str,"(inf, 0)",False,False,None,None
2,age,Integer,age,int,"(inf, 1)",False,False,None,None
3,updated,DateTime,updated,datetime,"(inf, 3)",False,False,None,utcnow
4,added,DateTime,added,datetime,"(inf, 4)",False,False,None,utcnow


We create this table just as we did the one before, and show the new schema using inspection.

In [16]:
with core.begin_ddl() as emitter:
    tab1 = emitter.create_table_if_not_exists(container_type=MyContainer1)

for ci in core.inspect_columns('mytable1'):
    print(ci)

{'name': 'id', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 1}
{'name': 'name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'age', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'updated', 'type': DATETIME(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'added', 'type': DATETIME(), 'nullable': True, 'default': None, 'primary_key': 0}


We can create a container instance just as we did before. Note that `id`, `updated`, and `added` are optionally now because we expect the database to create them.

In [17]:
o = MyContainer1(name='John Doe', age=30)
o

MyContainer1(name='John Doe', age=30, _id=MISSING, updated=MISSING, added=MISSING)

As expected, calling `.birthyear()` raises an exception because the `added` entry has not been recorded - that will happen at insertion into the db.

In [18]:
try:
    o.birthyear()
except AttributeError as e:
    print('error raised:', e)

error raised: Cannot calculate birthyear without the added date. Did you mean to call this on a retrieved container instance?


After inserting the object into the database and retrieving it again, we can see that those previously missing fields have been populated.

In [19]:
with tab1.query() as q:
    q.insert_single(o)
    results = q.select()
results[0]

MyContainer1(name='John Doe', age=30, _id=1, updated=datetime.datetime(2024, 5, 15, 18, 5, 11, 747969), added=datetime.datetime(2024, 5, 15, 18, 5, 11, 747971))

And now we can call the `birthyear()` method.

In [20]:
results[0].birthyear()

1994

### Conclusion

For more detailed explanations of these topics, see the documentation and API reference provided on the website. Good luck!